\begin{equation*}
\newcommand{\E}{\mathbb{E}}
\newcommand{\Nor}{\mathcal{N}}
\end{equation*}




# Due Dates

* Textbook: Tuesday, December 4. Give to Bolei, Keller 5-178
* Coding: Tuesday, December 4, at 11:59 PM

# Textbook Problem

* 4.34





In [ ]:
# This is code to load the assignment.
# You'll need to run this code do or restart the assignment.
from loadAssignment import loadAssignment
Assignment, Questions, Submit, Data = loadAssignment(10)

# These are modules that we need
# once you run this code, you don't need to load them again
import autograd.numpy as np
import autograd as ag
import scipy.linalg as la
import scipy.signal as sp
import scipy.stats as st
import numpy.random as rnd
import matplotlib.pyplot as plt
%matplotlib inline



# Question 0

This assignment will step through the application of the bootstrap particle filter for a simple
video tracking application.

The goal is to draw a box at location $(p_x,p_y)$ and size $s$ which
        tracks an object as it moves through the video. (In this case, you 
        will will make the box follow my hoodie. See the cells at the bottom.)

The state of the system is given by:
	
\begin{equation*}
x[k] = \begin{bmatrix}
p_x[k] \\
p_y[k] \\
s[k] \\
p_x[k-1] \\
p_y[k-1] \\
s[k-1]
\end{bmatrix}
\end{equation*}
        
Find matrices $A$ and $B$ such that:

\begin{equation*}
x[k+1] = A x[k] + Bw[k]
\end{equation*}

with the following properties:
       
* If $w[k]=0$, then the velocity of each of the coordinates remains constant:
\begin{equation*}
\begin{bmatrix}
p_x[k+1]-p_x[k] \\
p_y[k+1]-p_y[k] \\
s[k+1]-s[k]
\end{bmatrix} = 
\begin{bmatrix}
p_x[k]-p_x[k-1] \\
p_y[k]-p_y[k-1] \\
s[k]-s[k-1]
\end{bmatrix}
\end{equation*}


*  If $w[k]$ is a three dimensional Gaussian with 
$w[k]\sim \mathcal{N}(0,I)$, then the conditional 
covariance of $\begin{bmatrix}
p_x[k+1] \\
p_y[k+1] \\
s[k+1] 
\end{bmatrix}$ given $x[k]$ is given by 
\begin{equation*}
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 0.0004
\end{bmatrix}
\end{equation*}



In [ ]:
# Define A and B here

Questions[0].checkAnswer(A,B)


# Question 1

For the bootstrap particle filter, we sample from the distribution $p(x[k+1]|x[k])$. 

Write a function that generates a sample of $x[k+1]$ given $x[k]$,
assuming that $w[k] \sim \mathcal{N}(0,I)$:

Your function should have the form:

```
xNext = sampleDynamics(x)
```



In [ ]:
# Define your sampling function here


Questions[1].checkAnswer(sampleDynamics)


# Question 2



The bootstrap particle filter uses weights of the form:

\begin{equation*}
w_k^{(i)} \propto p(y_k | x_k^{(i)}),
\end{equation*}
where $p(y|x)$ is the measurement likelihood function, and $x_k^{(i)}$ is particle $i$.

If we have $N$ particles, the weights are normalized so that $\sum_{i=0}^{N-1} w_k^{(i)} = 1$.



Write a function of the form:

```
W = bootstrapWeights(X,y,likelihood)
```

Here the inputs to the function are:

* `X` - an $N \times n$ array of particles. Here $N$ is the number of  particles and $n$ is the state dimension.
* `y` - a measurement
* `likelihood` - a function so that `p(y|x) = likelihood(y,x)`. 

`W` should be a length $N$ array of weights corresponding to the particles from `X`.

(The reason we want this to work in general is that the actual likelihood function that we will use in the application is complicated. So it is easier to just treat that function as a black box, rather than worry about coding it up.)



In [ ]:
# Define your function here

Questions[2].checkAnswer(bootstrapWeights)


# Question 3

As discussed above, the bootstrap particle filter generates samples according to the transition
density, $p(x_{k+1}|x_k)$. This is simple to implement, but does not incorporate any information from the measuremnets.

To get information from the measurements, the particicles are resampled at each time-step. Specifically, given particles $x_k^{(0)},\ldots,x_k^{(N-1)}$ and corresponding weights, $w_k^{(0)},\ldots,w_k^{(N-1)}$, the resampled particles are given by

\begin{equation*}
x_k^{(I_0)},x_k^{(I_1)},\ldots,x_k^{(I_{N-1})},
\end{equation*}
where each index $I_j$ is an inpdendent sample from the categorical distribution defined by the weights. In other words, $I_j = i$ with probability $w_k^{(i)}$. 

Write a function of the form:

```
X_resampled = resample(X,W)
```

Here `X` is an $N\times n$ array of particles, and `W` is a length $N$ array of weights.
`X_resampled` should be an $N\times n$ array of particles resampled as described above.

For this problem [`rnd.choice`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html) can be used for the sampling.



In [ ]:
# Define your function here

Questions[3].checkAnswer(resample)




# Question 4

Putting everything together, the bootstrap particle filter can be summarized as follows:

* Initialze particles $x_0^{(i)} \sim p(x_0)$ for $i=0,\ldots,N-$.
* For each time $k$:
	* Compute the weights $w_k^{(i)} \propto p(y_k | x_k^{(i)})$
	* Resample the particles based on the weights, $w_k^{(i)}$
	* Compute the next particle values $x_{k+1}^{(i)} \sim p(x_{k+1}|x_k^{(i)})$

Write a function of the form:

```
XParticle = bootstrapPF(Y,init,dynSampler,likelihood,N)
```
that implements the bootstrap particle filter.

The inputs are:

* `Y` - a $T\times m$ array corresponding to $T$ measurements each of dimension $m$.
* `init` - a function such that `init(N)` returns an $N\times n$ array of samples from the initial distribution, $p(x_0)$
* `dynSampler` - a function such that `dynSampler(x)` draws a sample of $x' \sim p(x'|x)$. 
* `likelihood` - a function such that `likelihood(y,x)=p(y|x)`
* N - the number of particles

Here `XParticle` should be a $T\times N \times n$ array, where $T$ is the number of time steps, $N$ is the number of particles, and $n$ is the dimension of $x_k$. 



In [ ]:
# Define your function here

Questions[4].checkAnswer(bootstrapPF)




Now we will see how it does. The cells below set up a simple image tracking problem.



In [ ]:
# First we'll load the images and do some basic pre-processing
# This may take a few seconds
import matplotlib.image as mpimg
import matplotlib.colors as mpcol

ImRange = range(1,70)

Images = []
Hsv_Images = []
for k in ImRange:
    # This is the actual image
    img = mpimg.imread('video/both%d.png' % k)
    # Our likelihood function requires hue-saturation-value, rather than red-green-blue
    hsv = mpcol.rgb_to_hsv(img)
    
    Images.append(img)
    
    Hsv_Images.append(hsv)


# Next we
# Now we'll initialize the state by drawing a box around my hoodie

import videoProcessing as vp
img= Images[0]
img_hsv = Hsv_Images[0]
plt.figure(figsize=(10,20))
plt.imshow(img)


d = np.array([605,150])
s = 28

x0 = np.hstack([d,s,d,s])
imProc = vp.imageProcessor(d,s,img_hsv)

vp.drawBox(d,s)
plt.axis('off')

In [1]:
# Let's run the particle filter on the video data
# It typically works with 100 particles, but occasionally, it will not track
# Increasing the number of particles will improve the tracking, but it will run slowly.
# You'll know you've gotten it right if the boxes are still on my sweatshirt in the final frame

NumParticles = 100
init = lambda N : np.tile(x0,(N,1))
XParticles = bootstrapPF(Hsv_Images,init,sampleDynamics,imProc.measLikelihood,NumParticles)
XMean = np.mean(XParticles,axis=1)

fig,ax = plt.subplots(1,2,figsize=(15,10))
img = Images[-1]
ax[0].imshow(img)
plt.axis('off')
for part in range(NumParticles):
    xCur = XParticles[-1,part]
    vp.drawBox(xCur[:2],xCur[2],ax[0],linewidth=.5)


ax[1].imshow(img)
vp.drawBox(XMean[-1,:2],XMean[-1,2],ax[1])
plt.axis('off')

NameError: name 'bootstrapPF' is not defined

In [ ]:
# Now we'll make a video of the result
# Don't run this cell too many times, because it will make python run out of memory

import matplotlib.animation as animation

VidImages = []
fig,ax = plt.subplots(1)
for i,img in enumerate(Images):
    img_plot = plt.imshow(img)
    box = vp.drawBox(XMean[i,:2],XMean[i,2],ax,linewidth=3)
    VidImages.append([img_plot,box])
    
    
Anim = animation.ArtistAnimation(fig,VidImages,interval=100, blit=True,repeat=False)
Vid = Anim.to_jshtml()
import gc
fig.clf()
plt.close()
gc.collect()

In [ ]:
# Now let's watch it!
from IPython.display import HTML

HTML(Vid)


# Final Score

You can run this code to see all of your scores.




In [ ]:
Assignment.showResults()




# Submission

Save your work and run this cell to submit. It will only work if you have the internet.



In [ ]:
Submit()